In [3]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import grangercausalitytests
from vortexasdk import CargoTimeSeries, Products, Geographies

warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams.update({'font.size': 14})
pd.options.display.max_columns = None

- It's not all rosy. Global crude oil markets are navigating a complex landscape. This is largely due to geopolitical tensions and supply disruptions. There's a noticeable shift in global oil dynamics, with Brazil joining OPEC+ last year and tensions surrounding the petrodollar. 

- Vortexa data shows that... 

- In this notebook we analyse Vortexa's crude oil floating storage data (retrieved using our Python SDK) and its relationship to crude oil prices. Floating storage is directly determined by oil prices and market dynamics; however, it is not known whether floating storage may be predictive of oil prices in the future. In other words, can floating storage be used as a trading signal for crude oil? Here we apply Granger causality analysis on floating storage and price data, and for the first time demonstrate that floating storage may have significant predictive value for spot crude oil prices 2-3 months into the future.

- Fetching and Visualising Data

- Vortexa Global Crude Oil Floating Storage Data

- Continuing from our previous analysis, we will look at the 5 year period ending ono 31 January 2024. This period encapsulates the COVID pandemic, the conflict between Russia/Ukraine and the war between Israel and Palestine. 

In [ ]:
#Defining date range for historical analysis
START_DATE = datetime(2020, 1, 31)
END_DATE = datetime(2025, 1, 31)
DATE_RANGE = pd.date_range(START_DATE, END_DATE)

#Floating storage unit (t = metric tonnes, b = barrels)
UNIT = 't'